In [1]:
# Importing libraries
import gym
import numpy as np
from itertools import count
from collections import namedtuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
# Importing PyTorch here

In [2]:
for i in reversed(range(10)):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [3]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [4]:
env = gym.make('CartPole-v0') # We make the Cartpole environment here
# You can move either left or right to balance the pole
# Lets implement the Actor critic network
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) # 4 because there are 4 parameters as the observation space
        self.actor = nn.Linear(128, 2) # 2 for the number of actions
        self.critic = nn.Linear(128, 1) # Critic is always 1
        self.saved_actions = []
        self.rewards = []
    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

In [9]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()
# In this function, we decide whehter we want the block to move left or right,based on what the model decided

In [10]:
def finish_episode():
    # We calculate the losses and perform backprop in this function
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses =[]
    returns = []
    
    for r in model.rewards[::-1]:
        R = r # + 0.99 * R # 0.99 is our gamma number
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
        
        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
    
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    
    loss.backward()
    optimizer.step()
    
    del model.rewards[:]
    del model.saved_actions[:]

In [11]:
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [12]:
def train():
    running_reward = 10
    for i_episode in count(): # We need around this much episodes
        state = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0: # We will print some things out
            print("Episode {}\tLast Reward: {:.2f}\tAverage reward: {:.2f}".format(
                i_episode, ep_reward, running_reward
            ))
        if running_reward > env.spec.reward_threshold:
            print("Solved, running reward is now {} and the last episode runs to {} time steps".format(
                    running_reward, t
            ))
            break
            # This means that we solved cartpole and training is complete

In [13]:
train()

Episode 0	Last Reward: 17.00	Average reward: 10.35
Episode 10	Last Reward: 25.00	Average reward: 13.09
Episode 20	Last Reward: 28.00	Average reward: 19.79
Episode 30	Last Reward: 84.00	Average reward: 27.74
Episode 40	Last Reward: 33.00	Average reward: 38.05
Episode 50	Last Reward: 65.00	Average reward: 44.94
Episode 60	Last Reward: 24.00	Average reward: 47.27
Episode 70	Last Reward: 23.00	Average reward: 41.74
Episode 80	Last Reward: 27.00	Average reward: 41.94
Episode 90	Last Reward: 89.00	Average reward: 47.02
Episode 100	Last Reward: 74.00	Average reward: 46.85
Episode 110	Last Reward: 37.00	Average reward: 48.25
Episode 120	Last Reward: 26.00	Average reward: 46.83
Episode 130	Last Reward: 49.00	Average reward: 45.26
Episode 140	Last Reward: 38.00	Average reward: 46.60
Episode 150	Last Reward: 19.00	Average reward: 46.28
Episode 160	Last Reward: 55.00	Average reward: 46.31
Episode 170	Last Reward: 54.00	Average reward: 42.26
Episode 180	Last Reward: 59.00	Average reward: 43.59
Epis

In [ ]:
R = 0
returns = []
k = [{"reward":2},{"reward":6},{"reward":8},{"reward":12}]
for step in reversed(range(len(k))):
    R = k[step]['reward'] + 0.99 * R
    k[step]['target'] = R    


In [ ]:
print(returns)

[27.424388, 25.6812, 19.88, 12.0]


In [ ]:
print(k)

[{'reward': 2, 'target': 27.424388}, {'reward': 6, 'target': 25.6812}, {'reward': 8, 'target': 19.88}, {'reward': 12, 'target': 12.0}]
